In [ ]:
# Load map of Axial caldera showing locations of BOTPT instruments (red circles)
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-caldera-ed-sm.png")

In [ ]:
#Zoomed in map of Axial caldera
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-zoom-ed-sm.png")

### API Information Setup

In [ ]:
USERNAME = 'OOIAPI-J97520T1AYPUNI'
TOKEN =  'TEMP-TOKEN-YD01XSNDIO57MP'

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc
import csv

### Instrument Information 

In [ ]:
# Instrument Information RS03ASHS-MJ03B-09-BOTPTA304
subsite = 'RS03ASHS'
node = 'MJ03B'
sensor = '09-BOTPTA304'
method = 'streamed'
stream = 'botpt_nano_sample'
beginDT = '2018-04-04T10:01:01.000Z'
endDT = '2018-09-1T10:01:01.000Z'
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,
    'limit':20000,   #Investigate!!! What are the other possible parameters? What does this one do?? 
}

In [ ]:
# Grab the data
r = requests.get(data_request_url, params=params, auth=(USERNAME, TOKEN))
data = r.json()

In [ ]:
type(data)
# len(data)

### Pull in data 

In [ ]:
botpt_time = []
bottom_pressure = []
for i in range(len(data)):
    botpt_time.append(nc.num2date(data[i]['time'],'seconds since 1900-01-01').replace(microsecond=0))
    bottom_pressure.append(data[i]['bottom_pressure'])

### Make preliminary PSI plot 

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(botpt_time, bottom_pressure, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('psia')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()
type(bottom_pressure)

List Test Cell

In [ ]:
# testvalues = [1,1,1]
# b = [testvalues*3 for testvalues in testvalues]
# z = testvalues*mult
# print(b)
# l = [x * 2 for x in l]

### Psi to Depth

In [ ]:
# depths = bottom_pressure*.67
depths = [bottom_pressure*0.67 for bottom_pressure in bottom_pressure]
# depths = np.array([bottom_pressure])*.67
# print(d)
# cal_depths= depths.tolist()
type(depths)

### Make preliminary Depth plot

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.plot(botpt_time, depths, marker=".", markersize=1, linestyle=None)
fig.set_size_inches(28, 6)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_ylabel('depth')
plt.xlabel('Time')
plt.xticks(rotation=30)
plt.tight_layout()
# fig.subplots_adjust(hspace=0)
# plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()
# type(bottom_pressure)

Reading List in Pandas 

In [ ]:
df = pd.DataFrame(np.array(depths))
print (df)

## Remove Tides

### Loading Tide Data


In [ ]:
tide_file = './tide_data/pred_D.txt'
tides_D = pd.read_csv(tide_file, delim_whitespace=True, dtype=object)

In [ ]:
tides_D.head

In [ ]:
tides_D['datetime']=tides_D['year'] + '-' + tides_D['month'] + '-' + tides_D['day'] + \
            'T' + tides_D['hour'] + ':' + tides_D['minute'] + ':' + tides_D['second']

In [ ]:
tides_D.datetime.head 
len(ti)

In [ ]:
# loading data from a CSV file into a Pandas DataFrame
tides = pd.read_csv("pred_D.csv")
# tides.head()

# This works but data is too large
# with open('pred_D.csv') as csvfile:
#     readCSV = csv.reader(csvfile, delimiter=',')
#     for row in readCSV:
#         print(row)

### Create Output Directory

In [ ]:
# make the output directory
new_dir = '15s_tide_data/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

### Downsample Data

In [ ]:
# read in the data directly off THREDDS and write out as subsampled pickled pandas dataframe
num = 0
for i in datasets:
    print('Downsampling file {} of {}'.format(str(num + 1), str(len(datasets))))
    ds = xr.open_dataset(i)
#     ds = ds.swap_dims({'obs': 'time'})
    
    tides = pd.DataFrame()
    tides['pred_D.csv'] = ds['pred_D.csv'].to_pandas().resample('15S').mean()
    del botpt.index.name

#     tides = botpt.dropna()

    out = '15s_tide_data/' + i.split('/')[-1][:-3] + '_resampled' + '.pd'
    num = num +1

    with open(out, 'wb') as fh:
        pk.dump(botpt,fh)

    gc.collect()
print('Complete!')

In [ ]:
# create a single file with all the pickled data.
tides = pd.DataFrame()
for path, subdirs, files in os.walk('15s_tide_data/'):
#     files = files[1:]
    for name in files:
        file_name = os.path.join(path, name) 
        with open(file_name, 'rb') as f:
            pd_df = pk.load(f)
            tides = tides.append(pd_df)

tides.index.name = 'time'
tides = tides.sort_values(by=['time'])

with open('tides.pd', 'wb') as f:
    pk.dump(tides,f)

### Tides 

### Subtracting Tides from BOTPT 

### Detided BOTPT Plot